In [1]:
%pip install -qU google-generativeai python-dotenv pillow

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
from PIL import Image # 이미지 파일을 다루기 위한 라이브러리 추가

load_dotenv()
try:
    api_key = os.getenv("GOOGLE_API_KEY")
    if not api_key:
        raise ValueError("GOOGLE_API_KEY가 .env 파일에 없습니다.")
    genai.configure(api_key=api_key)
    print("Google API 키가 성공적으로 설정되었습니다.")
except Exception as e:
    print(f"API 키 설정에 문제가 있습니다: {e}")

Google API 키가 성공적으로 설정되었습니다.


c:\Users\user\anaconda3\envs\moit_agent_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# --- 셀 2: 멀티모달 모델 및 프롬프트 정의 ---

# 1. 멀티모달 모델(gemini-1.5-flash)을 준비합니다.
model = genai.GenerativeModel('gemini-2.5-flash')
print(f"멀티모달 모델 '{model.model_name}'이 준비되었습니다.")

# 2. AI에게 내릴 업무 지시서(프롬프트)를 작성합니다.
vision_prompt_text = """
당신은 사용자의 일상 사진과 간단한 설문 결과를 보고, 그 사람에게 가장 잘 어울릴 것 같은 새로운 취미를 창의적으로 추천하는 라이프스타일 큐레이터입니다.

[사용자 정보]
- 설문 답변: {survey}
- 아래 제공된 사진들

[지시사항]
1. 주어진 사진들 속 인물, 사물, 배경, 분위기를 종합적으로 분석하여 이 사람의 잠재적인 관심사와 성향을 추측하세요.
2. 설문 답변과 사진 분석 결과를 종합적으로 고려하여, 이 사람에게 가장 잘 어울릴 만한 취미 3가지를 추천해주세요.
3. 각 취미에 대해 "왜" 이 취미를 추천하는지, 사진 속 단서와 설문 결과를 근거로 설득력 있게 설명해주세요.
4. 최종 답변은 친절하고 따뜻한 말투의 추천 문장으로 작성해주세요.
"""

print("프롬프트가 준비되었습니다.")

멀티모달 모델 'models/gemini-2.5-flash'이 준비되었습니다.
프롬프트가 준비되었습니다.


In [8]:
# --- 셀 3: 실제 테스트 실행 ---

# 1. 테스트용 입력 데이터를 준비합니다.

# (예시) 간단한 설문 데이터
sample_survey = {
    "선호하는 활동": "실내보단 주로 밖에서 다양한 것을 보고 사람을 만나는 것을 즐김",
    "새로운 경험에 대한 태도": "익숙한 것을 더 좋아하지만, 가끔 새로운 시도도 즐김"
}

# (중요!) 테스트에 사용할 실제 이미지 파일들의 경로를 여기에 입력해주세요.
# 예: "C:/Users/user/Pictures/my_photo_1.jpg"
image_file_paths = [
    r"C:\Users\user\Documents\카카오톡 받은 파일\KakaoTalk_20251014_135916907_02.png",
    r"C:\Users\user\Documents\카카오톡 받은 파일\KakaoTalk_20251014_135916907.png",
]

# 2. AI에게 전달할 최종 입력 데이터를 구성합니다.
prompt = vision_prompt_text.format(survey=sample_survey)
image_parts = []

try:
    if not image_file_paths:
        print("분석할 이미지가 없습니다. 'image_file_paths' 리스트에 이미지 파일 경로를 추가해주세요.")
    else:
        for path in image_file_paths:
            img = Image.open(path)
            image_parts.append(img)
        
        # 3. 이미지와 프롬프트를 함께 모델에 전달하여 실행!
        print("\n--- AI가 사진을 분석하고 취미를 추천하는 중입니다... ---")
        response = model.generate_content([prompt] + image_parts)
        
        # 4. 최종 결과 출력
        print("\n--- AI의 최종 추천 결과 ---")
        print(response.text)

except FileNotFoundError as e:
    print(f"오류: 이미지 파일을 찾을 수 없습니다. 경로를 확인해주세요. ({e})")
except Exception as e:
    print(f"AI 실행 중 오류가 발생했습니다: {e}")


--- AI가 사진을 분석하고 취미를 추천하는 중입니다... ---

--- AI의 최종 추천 결과 ---
안녕하세요! 라이프스타일 큐레이터입니다. 보내주신 소중한 사진들과 설문 답변을 꼼꼼히 살펴보았습니다. 활기찬 에너지와 함께 세상과 소통하고 싶은 고객님의 멋진 모습이 느껴져 저도 즐거운 마음으로 분석했어요.

**[사진 및 설문 종합 분석]**

*   **사진 1 (헬스장):** 깔끔하고 잘 정돈된 헬스장의 모습에서 꾸준히 자기 관리를 하고 체력을 단련하는 것을 중요하게 생각하시는 분이라는 인상을 받았습니다. 운동을 통해 얻는 성취감과 건강을 추구하는 성향이 엿보여요.
*   **사진 2 (풋살장):** 여러 사람과 함께 활발하게 풋살 경기를 즐기는 모습이 인상적입니다. '실내보단 주로 밖에서 다양한 것을 보고 사람을 만나는 것을 즐김'이라는 설문 답변과 완벽하게 부합하는 사진이에요. 팀 스포츠를 통해 얻는 협동심과 경쟁의 즐거움, 그리고 탁 트인 공간에서의 활동을 선호하심을 알 수 있습니다.
*   **설문 답변:**
    *   **'실내보단 주로 밖에서 다양한 것을 보고 사람을 만나는 것을 즐김':** 야외 활동과 사회적 교류를 매우 중요하게 생각하시는군요. 새로운 풍경을 접하고 사람들과 어울리는 데서 큰 에너지를 얻으시는 분으로 보입니다.
    *   **'익숙한 것을 더 좋아하지만, 가끔 새로운 시도도 즐김':** 완전히 생소한 것보다는 익숙한 틀 안에서 변화를 주거나, 새로운 도전을 즐기는 균형 잡힌 성향을 가지고 계시네요.

이 모든 정보를 종합해 볼 때, 고객님께서는 활동적인 라이프스타일을 즐기며, 사람들과의 교류를 통해 에너지를 얻고, 새로운 환경을 탐험하는 것을 좋아하는 분으로 판단됩니다. 건강하고 활기찬 삶을 추구하면서도, 때로는 신선한 자극을 즐기는 성향이시죠!

---

이러한 분석을 바탕으로, 고객님의 일상에 더욱 활력과 즐거움을 더해줄 새로운 취미 3가지를 추천해 드립니다!

### **추천 취미 1: 도심 러닝 크루 가